# OpenAI Function Calling In LangChain

In [1]:
import openai
import os

In [2]:
openai.api_key = open("openAI_key.txt", "r").read().strip("\n")

In [3]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. 

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like this:

In [4]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [5]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [6]:
foo.name

'Joe'

In [7]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [8]:
foo.age

'bar'

In [9]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [10]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [11]:
foo_p.name

'Jane'

The next cell is expected to fail.

In [12]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  value is not a valid integer (type=type_error.integer)

In [13]:
class Class(BaseModel):
    students: List[pUser]

In [14]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [15]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition


In [16]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [17]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [18]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [19]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code',
    'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [20]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

Note: The next cell is expected to generate an error.

In [21]:
convert_pydantic_to_openai_function(WeatherSearch1)

KeyError: 'description'

In [22]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [23]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch2',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code', 'type': 'string'}},
  'required': ['airport_code']}}

In [24]:
from langchain.chat_models import ChatOpenAI

In [25]:
model = ChatOpenAI()

In [26]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}})

In [27]:
model_with_function = model.bind(functions=[weather_function])

In [28]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}})

## Forcing it to use a function

We can force the model to use a function

In [29]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [30]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}})

In [31]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "LAX"\n}'}})